In [1]:
# %pip install pytorch-msssim
# %pip install torchmetrics
# %pip install lpips
# %pip install h5py
# %pip install pathos
# %pip install multiprocess
# %pip install lmdb

In [2]:
!nvidia-smi

Thu Jul  3 04:28:06 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 576.52                 Driver Version: 576.52         CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1650 Ti   WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   55C    P5              5W /   50W |      89MiB /   4096MiB |     27%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
import torch
import os
import gc
import cv2
import lpips
import lmdb
import time
import shutil
import random
import torchmetrics
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt
import torch.multiprocessing as mp
import torchvision.utils as vutils
from tqdm import tqdm
from functools import partial
from torchvision import transforms
from torch.utils.data import Dataset
from pytorch_msssim import MS_SSIM
from multiprocess.pool import ThreadPool
from torch.cuda.amp import autocast, GradScaler
from torch.utils.data import DataLoader
from torchmetrics.image import PeakSignalNoiseRatio

c:\Users\Taneem\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
mp.set_start_method('spawn', force=True)

In [5]:
# Configuration
DEVICE = torch.device("cuda" if torch.cuda.is_available(
) else "mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {DEVICE}")
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.deterministic = False

COLORBLIND_TYPES = ["protanopia", "deuteranopia", "tritanopia"]
NUM_EPOCHS = 50
BATCH_SIZE = 16
IMG_SIZE = 256

Using device: cuda


In [6]:
def process_batch(file_batch, data_dir, cb_type, img_size):
    original_dir = os.path.join(data_dir, "original")
    type_dir = os.path.join(data_dir, cb_type)

    orig_batch = []
    cb_batch = []

    for fname in file_batch:
        try:
            orig_img = cv2.imread(os.path.join(original_dir, fname))
            if orig_img is None:
                raise FileNotFoundError(f"Original image {fname} not found")
            orig_img = cv2.cvtColor(orig_img, cv2.COLOR_BGR2RGB)
            orig_img = cv2.resize(orig_img, (img_size, img_size))

            cb_img = cv2.imread(os.path.join(type_dir, fname))
            if cb_img is None:
                raise FileNotFoundError(f"Colorblind image {fname} not found")
            cb_img = cv2.cvtColor(cb_img, cv2.COLOR_BGR2RGB)
            cb_img = cv2.resize(cb_img, (img_size, img_size))

            orig_batch.append(orig_img)
            cb_batch.append(cb_img)
        except Exception as e:
            print(f"\nError processing {fname}: {str(e)}")
            raise

    return np.array(orig_batch), np.array(cb_batch)


def preprocess_type(data_dir, cb_type, output_path, num_workers=6, batch_size=1000):
    """Preprocess and write LMDB for one colorblind type"""
    type_dir = os.path.join(data_dir, cb_type)
    files = [f for f in os.listdir(type_dir)
             if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    if not files:
        raise ValueError(f"No images found in {type_dir}")

    batches = [files[i:i + batch_size]
               for i in range(0, len(files), batch_size)]

    env = None
    try:
        # Estimate needed map size
        map_size = 15 * 1024 ** 3  # 15 GB max
        env = lmdb.open(output_path, map_size=map_size)

        with env.begin(write=True) as txn:
            with ThreadPool(num_workers) as p:
                with tqdm(total=len(batches), desc=f"Processing {cb_type}") as pbar:
                    process_func = partial(process_batch,
                                           data_dir=data_dir,
                                           cb_type=cb_type,
                                           img_size=IMG_SIZE)

                    total_idx = 0
                    for orig_data, cb_data in p.imap(process_func, batches):
                        for img_idx in range(len(orig_data)):
                            txn.put(f'original_{total_idx}'.encode(),
                                    orig_data[img_idx].tobytes())
                            txn.put(f'{cb_type}_{total_idx}'.encode(),
                                    cb_data[img_idx].tobytes())
                            total_idx += 1
                        pbar.update(1)

    except Exception as e:
        print(f"Error during preprocessing {cb_type}: {str(e)}")
        if os.path.exists(output_path):
            print(f"Deleting incomplete LMDB {output_path}")
            # Add retry loop for Windows
            for _ in range(3):
                try:
                    shutil.rmtree(output_path, ignore_errors=True)
                    break
                except PermissionError:
                    time.sleep(0.5)
        raise
    finally:
        if env is not None:
            env.close()
            # Add this for Windows compatibility
            if os.name == 'nt':
                try:
                    os.remove(os.path.join(output_path, 'lock.mdb'))
                except Exception as e:
                    pass

In [7]:
class ColorblindDataset(Dataset):
    def __init__(self, lmdb_path, cb_type, transform=None):
        self.lmdb_path = os.path.abspath(lmdb_path)
        self.cb_type = cb_type
        self.transform = transform or transforms.Compose([
            transforms.Normalize([0.5]*3, [0.5]*3)
        ])
        self._env = None  # Will be initialized per worker

        # Use temporary environment to count entries
        with lmdb.open(self.lmdb_path, readonly=True, lock=False) as env:
            with env.begin() as txn:
                self._len = txn.stat()['entries'] // 2
                if self._len == 0:
                    raise ValueError(f"LMDB {lmdb_path} contains 0 samples.")

    def _init_env(self):
        if self._env is None:
            self._env = lmdb.open(
                self.lmdb_path, readonly=True, lock=False, max_readers=128)

    def __getitem__(self, idx):
        try:
            self._init_env()
            with self._env.begin(buffers=True) as txn:
                orig_bytes = txn.get(f'original_{idx}'.encode())
                cb_bytes = txn.get(f'{self.cb_type}_{idx}'.encode())

                if orig_bytes is None or cb_bytes is None:
                    raise RuntimeError(f"Missing data for index {idx}")

                orig = np.frombuffer(orig_bytes, dtype=np.uint8).reshape(
                    IMG_SIZE, IMG_SIZE, 3)
                cb = np.frombuffer(cb_bytes, dtype=np.uint8).reshape(
                    IMG_SIZE, IMG_SIZE, 3)

            orig = torch.from_numpy(orig.copy()).permute(
                2, 0, 1).float().div(255)
            cb = torch.from_numpy(cb.copy()).permute(2, 0, 1).float().div(255)

            if self.transform:
                orig = self.transform(orig)
                cb = self.transform(cb)

            return orig, cb
        except Exception as e:
            print(f"Error loading index {idx}: {e}")
            raise

    def __len__(self):
        return self._len

    def close(self):
        if self._env is not None:
            self._env.close()
            self._env = None

In [8]:
# Autoencoder Model
class Autoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        # Encoder
        self.enc1 = nn.Sequential(
            nn.Conv2d(3, 32, 3, stride=2, padding=1),
            nn.BatchNorm2d(32), nn.ReLU(True))
        self.enc2 = nn.Sequential(
            nn.Conv2d(32, 64, 3, stride=2, padding=1),
            nn.BatchNorm2d(64), nn.ReLU(True))
        self.enc3 = nn.Sequential(
            nn.Conv2d(64, 128, 3, stride=2, padding=1),
            nn.BatchNorm2d(128), nn.ReLU(True))
        self.enc4 = nn.Sequential(
            nn.Conv2d(128, 256, 3, stride=2, padding=1),
            nn.BatchNorm2d(256), nn.ReLU(True))
        self.enc5 = nn.Sequential(
            nn.Conv2d(256, 512, 3, stride=2, padding=1),
            nn.BatchNorm2d(512), nn.ReLU(True))

        # Decoder with Skip Connections
        self.dec5 = nn.Sequential(
            nn.ConvTranspose2d(512, 256, 4, stride=2, padding=1),
            nn.BatchNorm2d(256), nn.ReLU(True))
        self.dec4 = nn.Sequential(
            nn.ConvTranspose2d(512, 128, 4, stride=2, padding=1),
            nn.BatchNorm2d(128), nn.ReLU(True))
        self.dec3 = nn.Sequential(
            nn.ConvTranspose2d(256, 64, 4, stride=2, padding=1),
            nn.BatchNorm2d(64), nn.ReLU(True))
        self.dec2 = nn.Sequential(
            nn.ConvTranspose2d(128, 32, 4, stride=2, padding=1),
            nn.BatchNorm2d(32), nn.ReLU(True))
        self.dec1 = nn.Sequential(
            nn.ConvTranspose2d(64, 3, 4, stride=2, padding=1),
            nn.Tanh())

    def forward(self, x):
        # Encoder
        e1 = self.enc1(x)  # 128
        e2 = self.enc2(e1)  # 64
        e3 = self.enc3(e2)  # 32
        e4 = self.enc4(e3)  # 16
        e5 = self.enc5(e4)  # 8

        # Decoder
        d5 = self.dec5(e5)
        d4 = self.dec4(torch.cat([d5, e4], 1))
        d3 = self.dec3(torch.cat([d4, e3], 1))
        d2 = self.dec2(torch.cat([d3, e2], 1))
        d1 = self.dec1(torch.cat([d2, e1], 1))
        return d1

In [9]:
# Training Utilities
class EarlyStopping:
    def __init__(self, patience=5, delta=0.001):
        self.patience = patience
        self.delta = delta
        self.counter = 0
        self.best_loss = float('inf')
        self.early_stop = False

    def __call__(self, val_loss):
        if (self.best_loss - val_loss) > self.delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True


def process_image(tensor):
    """Universal image processing function for proper denormalization"""
    img = tensor.detach().cpu().numpy()
    img = img.transpose(1, 2, 0)  # CHW to HWC
    img = np.clip(img, -1, 1)     # Ensure values are in [-1, 1]
    img = (img * 0.5 + 0.5)       # Scale to [0, 1]
    img = (img * 255).astype(np.uint8)
    return img

# Visualization with Saving


def visualize_and_save(orig, target, output, cb_type, epoch, batch_idx, save_dir):
    os.makedirs(save_dir, exist_ok=True)
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))

    for ax, img, title in zip(axes, [orig, target, output], ['Original', 'TargetDaltonized', 'Generated']):
        processed_img = process_image(img)
        ax.imshow(processed_img)
        ax.set_title(f"{title} ({cb_type})")
        ax.axis('off')

    plt.savefig(os.path.join(save_dir, f"{cb_type}_e{epoch+1}_b{batch_idx}.png"),
                bbox_inches='tight',
                dpi=100)
    plt.close()

In [10]:
def worker_init_fn(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

In [11]:
# Modified DataLoader setup
def get_loader(dataset, batch_size, num_workers, shuffle):
    return DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        num_workers=num_workers,
        pin_memory=True,
        worker_init_fn=worker_init_fn,
        persistent_workers=False
    )

In [12]:
def train_colorblind_type(cb_type):
    train_lmdb_path = f"temp_train_{cb_type}.lmdb"
    val_lmdb_path = f"temp_val_{cb_type}.lmdb"
    metric_path = f"metrics_{cb_type}.npz"
    checkpoint_path = f"checkpoint_{cb_type}.pth"

    try:
        # Check for existing checkpoint
        resume = os.path.exists(checkpoint_path)

        if not resume:
            # Fresh training setup
            print(f"\n{'='*40}\nPreprocessing {cb_type}\n{'='*40}")
            if not os.path.exists(train_lmdb_path):
                preprocess_type("data/train", cb_type, train_lmdb_path)
            if not os.path.exists(val_lmdb_path):
                preprocess_type("data/val", cb_type, val_lmdb_path)

            # Validate LMDBs only for fresh start
            for path in [train_lmdb_path, val_lmdb_path]:
                with lmdb.open(path, readonly=True) as env:
                    with env.begin() as txn:
                        entries = txn.stat()["entries"]
                        if entries == 0:
                            raise RuntimeError(f"LMDB {path} is empty!")
                        print(f"{path} has {entries//2} samples")

        # Initialize model and components
        model = Autoencoder().to(DEVICE)
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
        scaler = GradScaler()
        early_stopper = EarlyStopping(patience=5)
        ms_ssim = MS_SSIM(data_range=2.0).to(DEVICE)
        psnr = PeakSignalNoiseRatio().to(DEVICE)
        lpips_model = lpips.LPIPS(net='alex').to(DEVICE)

        start_epoch = 0
        metrics = {
            'train_loss': [], 'val_loss': [],
            'train_psnr': [], 'val_psnr': [],
            'train_lpips': [], 'val_lpips': []
        }

        # Resume logic
        if resume:
            print(f"\n{'='*40}\nResuming {cb_type} training\n{'='*40}")
            checkpoint = torch.load(checkpoint_path)

            model.load_state_dict(checkpoint['model_state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            scaler.load_state_dict(checkpoint['scaler_state_dict'])
            early_stopper.counter = checkpoint['early_stop_counter']
            start_epoch = checkpoint['epoch'] + 1
            metrics = checkpoint['metrics']

            print(f"Resuming from epoch {start_epoch} with:")
            print(f"- Best val loss: {min(metrics['val_loss']):.4f}")
            print(f"- Current early stop counter: {early_stopper.counter}")

        # Data loading (always needed)
        train_dataset = ColorblindDataset(train_lmdb_path, cb_type)
        val_dataset = ColorblindDataset(val_lmdb_path, cb_type)
        train_loader = get_loader(
            train_dataset, BATCH_SIZE, num_workers=0, shuffle=True)
        val_loader = get_loader(val_dataset, BATCH_SIZE,
                                num_workers=0, shuffle=False)

        # Training loop
        for epoch in range(start_epoch, NUM_EPOCHS):
            model.train()
            epoch_loss = epoch_psnr = epoch_lpips = 0.0

            # Training phase
            for batch_idx, (orig, target) in tqdm(enumerate(train_loader),
                                                  desc=f"{cb_type} Epoch {epoch+1}",
                                                  total=len(train_loader)):
                orig, target = orig.to(DEVICE), target.to(DEVICE)

                with autocast():
                    output = model(orig)
                    loss = 1 - ms_ssim(output, target)

                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()

                with torch.no_grad():
                    output_scaled = (output + 1) / 2
                    target_scaled = (target + 1) / 2
                    psnr_val = psnr(output_scaled, target_scaled)
                    lpips_val = lpips_model(output, target).mean()

                epoch_loss += loss.item()
                epoch_psnr += psnr_val.item()
                epoch_lpips += lpips_val.item()

                if batch_idx % 500 == 0:
                    visualize_and_save(orig[0], target[0], output[0], cb_type,
                                       epoch, batch_idx, "training_samples")

            # Update metrics
            metrics['train_loss'].append(epoch_loss / len(train_loader))
            metrics['train_psnr'].append(epoch_psnr / len(train_loader))
            metrics['train_lpips'].append(epoch_lpips / len(train_loader))

            # Validation phase
            model.eval()
            val_loss = val_psnr = val_lpips = 0.0
            with torch.no_grad():
                for orig, target in val_loader:
                    orig, target = orig.to(DEVICE), target.to(DEVICE)
                    output = model(orig)

                    loss = 1 - ms_ssim(output, target)
                    output_scaled = (output + 1) / 2
                    target_scaled = (target + 1) / 2

                    val_loss += loss.item()
                    val_psnr += psnr(output_scaled, target_scaled).item()
                    val_lpips += lpips_model(output, target).mean().item()

            metrics['val_loss'].append(val_loss / len(val_loader))
            metrics['val_psnr'].append(val_psnr / len(val_loader))
            metrics['val_lpips'].append(val_lpips / len(val_loader))

            # Save checkpoint
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scaler_state_dict': scaler.state_dict(),
                'early_stop_counter': early_stopper.counter,
                'metrics': metrics
            }, checkpoint_path)

            # Save best model
            if metrics['val_loss'][-1] == min(metrics['val_loss']):
                torch.save(model.state_dict(), f"best_model_{cb_type}.pth")
                visualize_and_save(orig[0], target[0], output[0], cb_type,
                                   epoch, 0, "best_results")

            # Early stopping check
            early_stopper(metrics['val_loss'][-1])
            if early_stopper.early_stop:
                print(f"Early stopping {cb_type} at epoch {epoch+1}")
                break

            # Print progress
            print(f"[Epoch {epoch+1}/{NUM_EPOCHS}] "
                  f"Train Loss: {metrics['train_loss'][-1]:.4f}, "
                  f"Val Loss: {metrics['val_loss'][-1]:.4f}, "
                  f"Train PSNR: {metrics['train_psnr'][-1]:.2f}, "
                  f"Val PSNR: {metrics['val_psnr'][-1]:.2f}, "
                  f"Train LPIPS: {metrics['train_lpips'][-1]:.4f}, "
                  f"Val LPIPS: {metrics['val_lpips'][-1]:.4f}")

        # Final cleanup after successful completion
        np.savez(metric_path, **metrics)
        torch.save(model.state_dict(), f"final_model_{cb_type}.pth")

        # Remove temporary files
        if os.path.exists(checkpoint_path):
            os.remove(checkpoint_path)

        print(f"\n{'='*40}\nTraining completed for {cb_type}\n{'='*40}")

    finally:
        # Cleanup resources without deleting LMDBs
        if 'train_dataset' in locals():
            train_dataset.close()
        if 'val_dataset' in locals():
            val_dataset.close()

        if 'train_loader' in locals():
            del train_loader
        if 'val_loader' in locals():
            del val_loader

        torch.cuda.empty_cache()
        gc.collect()

In [13]:
def force_delete(path):
    """Robust deletion with Windows lock handling"""
    for _ in range(5):  # 5 retries
        try:
            if os.path.exists(path):
                shutil.rmtree(path, ignore_errors=True)
                # Handle Windows lock file explicitly
                if os.name == 'nt':
                    lock_file = os.path.join(path, 'lock.mdb')
                    if os.path.exists(lock_file):
                        os.remove(lock_file)
                print(f"Successfully deleted {path}")
                return
        except Exception as e:
            print(f"Attempt {_+1}/5 failed for {path}: {str(e)}")
            time.sleep(1)

In [14]:
if __name__ == "__main__":
    os.makedirs("training_samples", exist_ok=True)
    os.makedirs("best_results", exist_ok=True)

    for cb_type in COLORBLIND_TYPES:
        train_colorblind_type(cb_type)
        current_files = [
            f"temp_train_{cb_type}.lmdb",
            f"temp_val_{cb_type}.lmdb"
        ]
        for path in current_files:
            print(f"\nCleaning up {path} before next type...")
            force_delete(path)

    print("\nTraining completed for all types!")


Preprocessing protanopia


Processing protanopia: 100%|██████████| 3/3 [00:23<00:00,  7.89s/it]


temp_train_protanopia.lmdb has 29670 samples
temp_val_protanopia.lmdb has 2967 samples
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


c:\Users\Taneem\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Taneem\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: c:\Users\Taneem\AppData\Local\Programs\Python\Python312\Lib\site-packages\lpips\weights\v0.1\alex.pth


...

[Epoch 26/50] Train Loss: 0.0208, Val Loss: 0.0176, Train PSNR: 27.48, Val PSNR: 27.37, Train LPIPS: 0.0466, Val LPIPS: 0.0422


tritanopia Epoch 27: 100%|██████████| 1855/1855 [18:25<00:00,  1.68it/s]


[Epoch 27/50] Train Loss: 0.0204, Val Loss: 0.0157, Train PSNR: 27.57, Val PSNR: 28.01, Train LPIPS: 0.0457, Val LPIPS: 0.0392


tritanopia Epoch 28: 100%|██████████| 1855/1855 [18:33<00:00,  1.67it/s]


Early stopping tritanopia at epoch 28

Training completed for tritanopia

Cleaning up temp_train_tritanopia.lmdb before next type...
Successfully deleted temp_train_tritanopia.lmdb

Cleaning up temp_val_tritanopia.lmdb before next type...
Successfully deleted temp_val_tritanopia.lmdb

Training completed for all types!


In [15]:
def plot_metrics(metrics, title_suffix="", figsize=(18, 6), dpi=300):
    plt.figure(figsize=figsize, dpi=dpi)

    # Loss Plot
    plt.subplot(1, 3, 1)
    plt.plot(metrics['train_loss'], 'b--', label='Train Loss')
    plt.plot(metrics['val_loss'], 'r-', label='Val Loss')
    plt.title('1 - MS-SSIM Loss', fontsize=12)
    plt.xlabel('Epoch', fontsize=10)
    plt.grid(True, alpha=0.3)
    plt.legend()

    # PSNR Plot
    plt.subplot(1, 3, 2)
    plt.plot(metrics['train_psnr'], 'g--', label='Train PSNR')
    plt.plot(metrics['val_psnr'], 'm-', label='Val PSNR')
    plt.title('PSNR (dB)', fontsize=12)
    plt.xlabel('Epoch', fontsize=10)
    plt.grid(True, alpha=0.3)
    plt.legend()

    # LPIPS Plot
    plt.subplot(1, 3, 3)
    plt.plot(metrics['train_lpips'], 'c--', label='Train LPIPS')
    plt.plot(metrics['val_lpips'], 'y-', label='Val LPIPS')
    plt.title('LPIPS (Lower = Better)', fontsize=12)
    plt.xlabel('Epoch', fontsize=10)
    plt.grid(True, alpha=0.3)
    plt.legend()

    plt.suptitle(f'Training Metrics {title_suffix}', y=1.02, fontsize=14)
    plt.tight_layout()

    save_path = f"training_metrics_{title_suffix.replace(' ', '_')}.png"
    plt.savefig(save_path, bbox_inches='tight')
    plt.close()
    print(f"Saved metrics plot to {save_path}")

In [16]:
for cb_type in COLORBLIND_TYPES:
    metrics = np.load(f"metrics_{cb_type}.npz")
    plot_metrics(
        metrics,
        title_suffix=f"({cb_type.capitalize()})",
        figsize=(20, 6),
        dpi=150
    )

Saved metrics plot to training_metrics_(Protanopia).png
Saved metrics plot to training_metrics_(Deuteranopia).png
Saved metrics plot to training_metrics_(Tritanopia).png


In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 16
colorblind_types = ["protanopia", "deuteranopia", "tritanopia"]

# Initialize metrics with correct data ranges
ms_ssim = MS_SSIM(data_range=2.0).to(device)  # For [-1, 1] range
psnr = PeakSignalNoiseRatio().to(device)
lpips_model = lpips.LPIPS(net='alex').to(device).eval()  # LPIPS in eval mode
ssim = torchmetrics.StructuralSimilarityIndexMeasure(
    data_range=2.0).to(device)  # Corrected data range


for cb_type in colorblind_types:
    print(f"\n{'='*40}\nTesting for {cb_type.upper()}\n{'='*40}")

    test_lmdb_path = f"temp_test_{cb_type}.lmdb"
    if not os.path.exists(test_lmdb_path):
        preprocess_type("data/test", cb_type, test_lmdb_path)

    test_dataset = ColorblindDataset(test_lmdb_path, cb_type)
    test_loader = DataLoader(
        test_dataset, batch_size=batch_size, num_workers=0,
        shuffle=False, pin_memory=True)

    model = Autoencoder().to(device)
    model_path = f"best_model_{cb_type}.pth"
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    # Initialize metrics
    metrics = {
        'loss': 0.0,
        'psnr': 0.0,
        'lpips': 0.0,
        'ssim': 0.0,
        'times': []
    }

    save_dir = f"test_results/{cb_type}"
    os.makedirs(save_dir, exist_ok=True)
    img_counter = 0  # For sequential image naming

    with torch.no_grad():
        for batch_idx, (orig, target) in enumerate(tqdm(test_loader, desc=f"Testing {cb_type}")):
            orig, target = orig.to(device, non_blocking=True), target.to(
                device, non_blocking=True)

            # Synchronize CUDA for accurate timing
            if device.type == 'cuda':
                torch.cuda.synchronize()
            start = time.time()

            output = model(orig)

            if device.type == 'cuda':
                torch.cuda.synchronize()
            metrics['times'].append(time.time() - start)

            # Calculate metrics
            metrics['loss'] += (1 - ms_ssim(output, target)).item()
            metrics['ssim'] += ssim(output, target).item()

            # For PSNR: scale to [0, 1]
            output_scaled = (output + 1) / 2
            target_scaled = (target + 1) / 2
            metrics['psnr'] += psnr(output_scaled, target_scaled).item()

            # LPIPS calculation (keep in [-1, 1] range)
            metrics['lpips'] += lpips_model(output, target).mean().item()

            # Save first 100 images for comparison
            if img_counter < 100:
                for i in range(output.size(0)):
                    visualize_and_save(
                        orig[i],
                        target[i],
                        output[i],
                        cb_type,
                        epoch=0,  # Use 0 for testing phase
                        batch_idx=img_counter,
                        save_dir=save_dir
                    )

                    # Save individual images
                    orig_img = process_image(orig[i])
                    gen_img = process_image(output[i])

                    # Convert to BGR for OpenCV and save
                    cv2.imwrite(
                        os.path.join(
                            save_dir, f"original_{img_counter:04d}.png"),
                        cv2.cvtColor(orig_img, cv2.COLOR_RGB2BGR)
                    )
                    cv2.imwrite(
                        os.path.join(
                            save_dir, f"generated_{img_counter:04d}.png"),
                        cv2.cvtColor(gen_img, cv2.COLOR_RGB2BGR)
                    )

                    img_counter += 1

    # Calculate final metrics
    n_batches = len(test_loader)
    print(f"\n{cb_type.upper()} Test Results:")
    print(f"- Avg MS-SSIM Loss: {metrics['loss'] / n_batches:.4f}")
    print(f"- Avg SSIM: {metrics['ssim'] / n_batches:.4f}")
    print(f"- Avg PSNR: {metrics['psnr'] / n_batches:.2f} dB")
    print(f"- Avg LPIPS: {metrics['lpips'] / n_batches:.4f}")

    # Timing statistics
    avg_time = np.mean(metrics['times']) * 1000
    fps = batch_size / (avg_time / 1000)
    print(f"- Inference Speed: {avg_time:.2f} ms/batch ({fps:.1f} FPS)")

    if 'test_dataset' in locals():
        test_dataset.close()

    current_file = f"temp_test_{cb_type}.lmdb"
    print(f"\nCleaning up {current_file} before next type...")
    force_delete(current_file)

    torch.cuda.empty_cache()

print("\nAll tests completed. Results saved in test_results/ directory.")

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: c:\Users\Taneem\AppData\Local\Programs\Python\Python312\Lib\site-packages\lpips\weights\v0.1\alex.pth


c:\Users\Taneem\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchmetrics\utilities\prints.py:62: FutureWarning: Importing `StructuralSimilarityIndexMeasure` from `torchmetrics` was deprecated and will be removed in 2.0. Import `StructuralSimilarityIndexMeasure` from `torchmetrics.image` instead.
  _future_warning(



Testing for PROTANOPIA


Testing protanopia: 100%|██████████| 186/186 [01:22<00:00,  2.26it/s]



PROTANOPIA Test Results:
- Avg MS-SSIM Loss: 0.0113
- Avg SSIM: 0.9125
- Avg PSNR: 28.39 dB
- Avg LPIPS: 0.0478
- Inference Speed: 52.73 ms/batch (303.4 FPS)

Cleaning up temp_test_protanopia.lmdb before next type...
Successfully deleted temp_test_protanopia.lmdb

Testing for DEUTERANOPIA


Testing deuteranopia: 100%|██████████| 186/186 [01:21<00:00,  2.29it/s]



DEUTERANOPIA Test Results:
- Avg MS-SSIM Loss: 0.0109
- Avg SSIM: 0.9174
- Avg PSNR: 29.51 dB
- Avg LPIPS: 0.0382
- Inference Speed: 52.59 ms/batch (304.2 FPS)

Cleaning up temp_test_deuteranopia.lmdb before next type...
Successfully deleted temp_test_deuteranopia.lmdb

Testing for TRITANOPIA


Testing tritanopia: 100%|██████████| 186/186 [01:21<00:00,  2.28it/s]



TRITANOPIA Test Results:
- Avg MS-SSIM Loss: 0.0157
- Avg SSIM: 0.9041
- Avg PSNR: 27.98 dB
- Avg LPIPS: 0.0392
- Inference Speed: 55.71 ms/batch (287.2 FPS)

Cleaning up temp_test_tritanopia.lmdb before next type...
Successfully deleted temp_test_tritanopia.lmdb

All tests completed. Results saved in test_results/ directory.
